In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot

In [0]:
x_train = pd.read_csv("/content/drive/My Drive/house price/x_train.csv")

In [3]:
x_train.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,Fireplaces,FullBath,GarageArea,GarageCars,GarageYrBlt,GrLivArea,HalfBath,KitchenAbvGr,LotArea,LotFrontage,LowQualFinSF,MSSubClass,MasVnrArea,MiscVal,MoSold,OpenPorchSF,OverallCond,OverallQual,PoolArea,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,FV,RH,RL,RM,...,No,BLQ,GLQ,LwQ,Rec,Unf,GasA,GasW,Grav,OthW,Wall,Y,FuseF,FuseP,Mix,SBrkr,Maj2,Min1,Min2,Typ,Attchd,Basment,BuiltIn,CarPort,Detchd,RFn,P,CWD,Con,ConLD,ConLI,ConLw,New,Oth,WD,AdjLand,Alloca,Family,Normal,Partial
0,856,854,0,3,706,0,1,0,150,0,0,2,548,2,2003.0,1710,1,1,8450,65.0,0,60,196.0,0,2,61,5,7,0,0,8,856,0,2003,2003,2008,0,0,1,0,...,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,1262,0,0,3,978,0,0,1,284,0,1,2,460,2,1976.0,1262,0,1,9600,80.0,0,20,0.0,0,5,0,8,6,0,0,6,1262,298,1976,1976,2007,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2,920,866,0,3,486,0,1,0,434,0,1,2,608,2,2001.0,1786,1,1,11250,68.0,0,60,162.0,0,9,42,5,7,0,0,6,920,0,2001,2002,2008,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3,961,756,0,3,216,0,1,0,540,272,1,1,642,3,1998.0,1717,0,1,9550,60.0,0,70,0.0,0,2,35,5,7,0,0,7,756,0,1915,1970,2006,0,0,1,0,...,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,1145,1053,0,4,655,0,1,0,490,0,1,2,836,3,2000.0,2198,1,1,14260,84.0,0,60,350.0,0,12,84,5,8,0,0,9,1145,192,2000,2000,2008,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [0]:
y_train = pd.read_csv("/content/drive/My Drive/house price/y_train.csv")

In [5]:
y_train.head()

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000


In [0]:
x_test = pd.read_csv("/content/drive/My Drive/house price/x_test.csv")

In [0]:
XX = x_train.to_numpy()
YX = y_train.to_numpy()
XY = x_test.to_numpy()

In [0]:
YY = YX.reshape(1460,)

In [21]:
from sklearn.linear_model import LinearRegression, RidgeCV
ridge = RidgeCV(alphas = [0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1, 3, 6, 7, 8, 9, 10, 30, 60])
ridge.fit(XX, YX)
alpha = ridge.alpha_
print("Best alpha :", alpha)

print("Trying again for more precision alpha values " + str(alpha))
ridge = RidgeCV(alphas = [alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, alpha * .85, 
                          alpha * .9, alpha * .95, alpha, alpha * 1.05, alpha * 1.1, alpha * 1.15,
                          alpha * 1.25, alpha * 1.3, alpha * 1.35, alpha * 1.4], cv = 10)

ridge_fit = ridge.fit(XX, YY)
alpha = ridge.alpha_
print("Best alpha :", alpha)
ridge_predictions = ridge.predict(XY)

Best alpha : 6.0
Trying again for more precision alpha values 6.0
Best alpha : 8.399999999999999


In [22]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
lasso = Lasso(max_iter = 5000, alpha = 11.2)
lasso_fit = lasso.fit(XX, YY)
lasso_predictions = lasso.predict(XY)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 381956019921.2762, tolerance: 920791133.4609975
  positive)


In [0]:
import xgboost as xgb
gbm = xgb.XGBRegressor(
                 colsample_bytree=0.1,   #ratio_of_constructing_each_tree
                 gamma=0.0,              #loss_reduction_param
                 learning_rate=0.01,     #for_updating_param
                 max_depth=3,            #maximum_depth_of_tree
                 min_child_weight=0,     #minimum_sum_of_child_weight
                 n_estimators=10000,     #total_no_of_iterations                                                                   
                 reg_alpha=0.0006,       #updating_coefficient_L1_regularization_term
                 reg_lambda=0.6,         #updating_coefficient_L2_regularization_term
                 subsample=0.7,          #sampling_training_data_randomly
                 seed=30,                #random_number_seed
                 silent=1)               #occurence_of_message
gbm_fit = gbm.fit(XX, YY)
gbm_predictions = gbm.predict(XY)

In [0]:
import lightgbm as lgb
lgb = lgb.LGBMRegressor(objective='regression',     
                        num_leaves=5,                  #num_of_leaves in a tree
                        learning_rate=0.05,            #updating_weights_to_mimize_loss
                        n_estimators=5000,             #num_of_iterations
                        max_bin = 55,                  #num_of_bins_binning_refers_to_continous_unique_value
                        bagging_fraction = 0.8,        #select_random_data_samples
                        bagging_freq = 5,              #perform_bagging_at_k_iteration
                        feature_fraction = 0.2319,     #randomly_select_feature_here_23%
                        feature_fraction_seed=9,       #random_seed_for_feature_fraction
                        bagging_seed=9,                #random_seed_for_bagging
                        min_data_in_leaf =6,           #minimum_no_leaf
                        min_sum_hessian_in_leaf = 11)
lgb = lgb.fit(XX, YY)
lgb_predictions = lgb.predict(XY)

In [0]:
from mlxtend.regressor import StackingCVRegressor
stack_gen = StackingCVRegressor(regressors=(lgb,lasso,ridge),
                                meta_regressor=gbm,
                                use_features_in_secondary=True)


In [26]:
stack_gen.fit(XX,YY)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 294473812675.31274, tolerance: 673724696.8072952
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 264241104748.64517, tolerance: 737691490.9028124
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 303519259926.4163, tolerance: 730758198.0951958
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 316614481915.3

StackingCVRegressor(cv=5,
                    meta_regressor=XGBRegressor(base_score=0.5,
                                                booster='gbtree',
                                                colsample_bylevel=1,
                                                colsample_bynode=1,
                                                colsample_bytree=0.1, gamma=0.0,
                                                importance_type='gain',
                                                learning_rate=0.01,
                                                max_delta_step=0, max_depth=3,
                                                min_child_weight=0,
                                                missing=None,
                                                n_estimators=10000, n_jobs=1,
                                                nthread=None,
                                                objective='reg:linear',
                                                random_state=0,
 

In [0]:
stack_pred = stack_gen.predict(XY)

In [32]:
len(stack_pred)

1459

In [0]:
sample = pd.read_csv("/content/drive/My Drive/house price/sample_submission.csv")#sample submission file

In [34]:
sample.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [38]:
best = pd.read_csv('/content/drive/My Drive/house price/sample_submission2.csv')#my last submission file of ANN
blend = stack_pred*0.50+best.SalePrice.values*0.50
blend

array([127773.020625  , 165165.0328125 , 188465.7053125 , ...,
       181458.8996875 , 120676.32546875, 220033.9278125 ])

In [0]:
sample['SalePrice'] = blend

In [40]:
sample.head()

,Id,SalePrice
0,1461,127773.020625
1,1462,165165.032812
2,1463,188465.705313
3,1464,196813.781250
4,1465,184219.404688


In [0]:
sample.to_csv("/content/drive/My Drive/house price/final.csv")